In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras import Sequential
from keras.utils.np_utils import to_categorical
from keras.layers import Dense
from keras.layers import LeakyReLU
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
import warnings
from sklearn.metrics import roc_curve, auc

In [2]:
df = pd.read_csv("compare.csv")

In [3]:
X = df.drop(["home_win"], axis=1)
y = df.home_win.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=49)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
# adjust format of inputs for neural network classifier
predictors = np.matrix(X_train)
target = to_categorical(y_train)
predictors_test = np.matrix(X_test)
target_test = to_categorical(y_test)

In [5]:
warnings.filterwarnings("ignore")

In [ ]:
# define parameter space for optimization
space = {
    "n_layers" : hp.randint("n_layers",62)
    "layer_types" : hp.choice("activation_function",["relu","sigmoid","softmax","softplus","softsign","tanh","selu","elu","exponential"])
    "nodes_per_layer" : hp.randint("nodes_per_layer",1000)
    "regularization" : hp.()
    "learning_rate" : hp.uniform("learning_rate",0.001,0.999)
    "" : hp.()
    "" : hp.()
    "" : hp.()
    "" : hp.()
    "" : hp.()
    "" : hp.()
}

In [ ]:
# define objective function to minimize
def neural_network_tuning(params):
    clf = create_model(params)
    acc = cross_val_score(clf, X, y, scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [98]:
# function to create model, required for KerasClassifier
def create_model(params):
    model = Sequential()
    model.add(Dense(10,activation='relu',input_shape=(14,)))
    #model.add(Dense(10, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [14]:
model = Sequential()
model.add(Dense(10,activation='relu',input_shape=(14,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [99]:
model = KerasClassifier(build_fn=create_model, verbose=1)

In [101]:
batch_size = [10,25,50,75,100]
epochs = [10,50,100]
param_grid = dict(batch_size=batch_size,epochs=epochs)
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=5)
grid_result=grid.fit(X,y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
228/228 [==============================] - 1s 3ms/step - loss: 140.5502 - accuracy: 0.5223
Epoch 2/50
228/228 [==============================] - 1s 3ms/step - loss: 54.6223 - accuracy: 0.5317
Epoch 3/50
228/228 [==============================] - 1s 3ms/step - loss: 37.4461 - accuracy: 0.5277
Epoch 4/50
228/228 [==============================] - 1s 3ms/step - loss: 20.3501 - accuracy: 0.5324
Epoch 5/50
228/228 [==============================] - 1s 3ms/step - loss: 15.7459 - accuracy: 0.5371
Epoch 6/50
228/228 [==============================] - 1s 3ms/step - loss: 9.8693 - accuracy: 0.5535
Epoch 7/50
228/228 [==============================] - 1s 3ms/step - loss: 6.6958 - accuracy: 0.5491
Epoch 8/50
228/228 [==============================] - 1s 3ms/step - loss: 5.7130 - accuracy: 0.5485
Epoch 9/50
228/228 [==============================] - 1s 3ms/step - loss: 4.1487 - accuracy: 0.5524
Epoch 10/50
228/228 [==============================] - 1s 3ms/step - loss: 5.1739 - accuracy: 

In [45]:
# initial XGBoost
import xgboost as xgb
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
xgb_model.fit(X_train,y_train)
y_pred = xgb_model.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.6193503400597984

In [5]:
# initial LinearSVC
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc.fit(X_train,y_train)
y_pred = lsvc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

C:\Users\abewo\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6251285287445812

In [6]:
# still higher iterations LinearSVC
lsvc = LinearSVC(max_iter=10000)
lsvc.fit(X_train,y_train)
y_pred = lsvc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

C:\Users\abewo\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6245728871352025

In [7]:
# still higher iterations LinearSVC
lsvc = LinearSVC(max_iter=15000)
lsvc.fit(X_train,y_train)
y_pred = lsvc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.6257251874557984

In [8]:
# still higher iterations LinearSVC
lsvc = LinearSVC(max_iter=17000)
lsvc.fit(X_train,y_train)
y_pred = lsvc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.6257251874557984

In [9]:
# even higher iterations LinearSVC
lsvc = LinearSVC(max_iter=25000)
lsvc.fit(X_train,y_train)
y_pred = lsvc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.6257251874557984

In [10]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier()
knc.fit(X_train,y_train)
y_pred = knc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.5851692095571376

In [11]:
# three neighbors
knc = KNeighborsClassifier(n_neighbors=3)
knc.fit(X_train,y_train)
y_pred = knc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.5774415620942874

In [12]:
# ten neighbors
knc = KNeighborsClassifier(n_neighbors=10)
knc.fit(X_train,y_train)
y_pred = knc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.6029283408302065

In [13]:
# twenty neighbors
knc = KNeighborsClassifier(n_neighbors=20)
knc.fit(X_train,y_train)
y_pred = knc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.61403824322479

In [14]:
# 100 neighbors
knc = KNeighborsClassifier(n_neighbors=100)
knc.fit(X_train,y_train)
y_pred = knc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.6221657437394146

In [15]:
# 200 neighbors
knc = KNeighborsClassifier(n_neighbors=200)
knc.fit(X_train,y_train)
y_pred = knc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.6127198363799784

In [16]:
# 300 neighbors
knc = KNeighborsClassifier(n_neighbors=300)
knc.fit(X_train,y_train)
y_pred = knc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

0.6134162354351074

In [17]:
# for good measure, running a non-linear SVC
# initial SVC

from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train,y_train)
y_pred = svc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Best SVC Model Roc " + str(round(roc_auc,3)))

Best SVC Model Roc 0.615


In [18]:
from sklearn.ensemble import BaggingClassifier
bagknc = BaggingClassifier(KNeighborsClassifier())
bagknc.fit(X_train,y_train)
y_pred=bagknc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Best bagknc Model Roc " + str(round(roc_auc,3)))

Best bagknc Model Roc 0.591


In [19]:
# bagknc, with better neighbors estimator from earlier
from sklearn.ensemble import BaggingClassifier
bagknc = BaggingClassifier(KNeighborsClassifier(n_neighbors=200))
bagknc.fit(X_train,y_train)
y_pred=bagknc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Best bagknc Model Roc " + str(round(roc_auc,3)))

Best bagknc Model Roc 0.614


In [20]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Initial RFC Model Roc " + str(round(roc_auc,3)))

Initial RFC Model Roc 0.618


In [21]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier()
abc.fit(X_train,y_train)
y_pred = abc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Initial abc ROC: " + str(round(roc_auc,3)))

Initial abc ROC: 0.628


In [25]:
# more estimators for abc
abc = AdaBoostClassifier(n_estimators=500)
abc.fit(X_train,y_train)
y_pred = abc.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Increased Estimators abc ROC: " + str(round(roc_auc,3)))

Increased Estimators abc ROC: 0.628


In [26]:
# initial Gradient Boosting Classifier+
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(X_train,y_train)
y_pred = gb.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Initial gb ROC: " + str(round(roc_auc,3)))

Initial gb ROC: 0.635


In [27]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hbgb = HistGradientBoostingClassifier()
hbgb.fit(X_train,y_train)
y_pred = hbgb.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Initial hbgb ROC: " + str(round(roc_auc,3)))

Initial hbgb ROC: 0.621


In [34]:
# instantiate Hard Voting Classifier

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = GaussianNB()

eclf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

In [35]:
for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring="accuracy")
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.65 (+/- 0.01) [Logistic Regression]
Accuracy: 0.64 (+/- 0.01) [Random Forest]
Accuracy: 0.65 (+/- 0.00) [naive Bayes]
Accuracy: 0.66 (+/- 0.00) [Ensemble]


In [36]:
eclf.fit(X_train,y_train)
y_pred = eclf.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Hard voting initial ROC: " + str(round(roc_auc,3)))

Hard voting initial ROC: 0.629


In [37]:
# soft voting classifier
clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = GaussianNB()

eclf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')

In [38]:
for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train, cv=5)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.65 (+/- 0.01) [Logistic Regression]
Accuracy: 0.64 (+/- 0.01) [Random Forest]
Accuracy: 0.65 (+/- 0.00) [naive Bayes]
Accuracy: 0.65 (+/- 0.00) [Ensemble]


In [39]:
eclf.fit(X_train,y_train)
y_pred = eclf.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Soft voting initial ROC: " + str(round(roc_auc,3)))

Soft voting initial ROC: 0.632


In [34]:
from sklearn.ensemble import StackingClassifier
estimators = [('abc', AdaBoostClassifier()),
             ('lsvc', LinearSVC(max_iter=25000)),
              ('rfc', RandomForestClassifier())]
sclf = StackingClassifier(estimators=estimators, final_estimator=GradientBoostingClassifier())

In [35]:
sclf.fit(X_train,y_train)
y_pred = sclf.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("Stacked Classifier initial ROC: " + str(round(roc_auc,3)))

Stacked Classifier initial ROC: 0.629


In [36]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=49).fit(X_train,y_train)
y_pred = clf.predict(X_test)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print("LogReg Initial ROC: " + str(round(roc_auc,3)))

LogReg Initial ROC: 0.625


So based on the initial AUROC scores for the various models, I am inclined to make and tune the following models:

Neural Network
GradientBoostingClassifier
Logistic Regression

After that, I will build a soft voting classifier of these and see how it does.